In [1]:
import json
import os 

import numpy as np

from tqdm import tqdm
from glob import glob

from norfair import Detection, Tracker, FilterSetup

In [2]:
def euclidean_distance(detection, tracked_object):
    detection_center = detection.points[0, :]
    tracked_center   = tracked_object.estimate[0,:]
    return np.linalg.norm(detection_center - tracked_center)

def get_coordinates(candidate):
    xmin,ymin,w,h = candidate[2:6]
    xc=int(xmin+w/2)
    yc=int(ymin+h/2)
    return np.array([[xc, yc],
                     [xmin, ymin],
                     [w, h]])

In [3]:
def track_dir(det_path, out_path, tracker, W=400, H=320):
    anns = [x.rstrip().split(',') for x in open(det_path)]
    anns = [list(map(float, lst)) for lst in anns]
    anns.sort(key=lambda x: x[0])
    
    frames = set([x[0] for x in anns])
    
    mot_objs = []
    for frame in frames:
        frame_objects = [x for x in anns if x[0]==frame]
        frame_objects = [x for x in frame_objects]
        detections = [Detection(get_coordinates(candidate), data=candidate) 
                      for candidate in frame_objects]
        tracked_objects = tracker.update(detections=detections)
        
        if len(tracked_objects) == 0:
            continue

        for tracked_object in tracked_objects:
            if not tracked_object.live_points.any():
                continue

            
            bbox = [int(tracked_object.estimate[1,0]),int(tracked_object.estimate[1,1]),
                    int(tracked_object.estimate[2,0]),int(tracked_object.estimate[2,1])]
            if len(tracked_object.last_detection.data) == 9:
                adds = tracked_object.last_detection.data[-3:]
                adds = [int(x) for x in adds[-3:]]
            elif len(tracked_object.last_detection.data) == 10:
                adds = tracked_object.last_detection.data[-4:]
                adds = [adds[-4]]+[int(x) for x in adds[-3:]]
            else:
                print('Invalid format')
                break
            
            mot_obj = [int(frame), tracked_object.id] + bbox + adds
            mot_objs.append(mot_obj)
    
    with open(out_path, 'w') as f:
        for obj in mot_objs:
            line = ','.join([str(x) for x in obj])
            f.write('%s\n' % line)

In [4]:
for det_path in tqdm(glob('./results/011-best-val/detection/*.txt')):
    tracker = Tracker(distance_function=euclidean_distance, 
                      distance_threshold=20,
                      hit_inertia_min=8,
                      hit_inertia_max=15,
                      initialization_delay=1,
                      filter_setup=FilterSetup(R=6,Q=0.1,P=15)) 
        
    seq = os.path.splitext(det_path.split(os.sep)[-1])[0]
    out_path = os.path.join('./results/011-best-val/norfair', '%s.txt' % seq)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    track_dir(det_path, out_path, tracker)

100%|███████████████████████████████████████████████| 13/13 [03:21<00:00, 15.50s/it]
